# Practicum 4: Reed-Solomon codes

<u> Naam: </u> 

In deze notebook zullen we trachten de Reed-Solomon code te implementeren. 

## Matrices en veeltermen over eindige velden
    
Voor dit practicum heb je opnieuw Gauss eliminatie over het veld $\mathbb{Z}_p$ nodig. Je kan deze ofwel kopieren uit practicum 3, ofwel de volgende implementatie gebruiken.  Deze heeft als input en output een numpy-array $M$ van gehele getallen.

<b>Opmerking:</b> In practicum 3 is het de bedoeling Gauss eliminatie zelf te implementeren, zonder de ingebouwde sympy-functie $\texttt{rref()}$. Het is dus niet de bedoeling de onderstaande functie te kopieren naar practicum 3.

In [1]:
import numpy as np
from sympy.polys.matrices import DomainMatrix
from sympy import GF

def gauss(M,p):
    #Zet M op naar een sympy DomainMatrix over het eindig veld Z_p
    F=GF(p)
    M=np.vectorize(lambda t: F(t))(M)
    M=DomainMatrix(M.tolist(), M.shape, F)
    #Pas de sympy-functie rref() toe; deze werkt ook over eindige velden
    M=M.rref()[0]
    #Zet M terug om naar een numpy array
    M=np.array(M.to_list())
    M=np.vectorize(lambda t: int(t) % p)(M)
    return M

Om te rekenen met veeltermen kan je bijvoorbeeld de ingebouwde implementatie in sympy gebruiken. Een voorbeeld:

In [2]:
from sympy import Poly
from sympy import symbols
x=symbols('x')

a=Poly([3,4,4,2],x,modulus=5)
print(a.all_coeffs())
a

[-2, -1, -1, 2]


Poly(-2*x**3 - x**2 - x + 2, x, modulus=5)

Deling met rest van veeltermen kan je nu gewoon doen met de operaties $\verb|\\|$ en $\verb|%|$:

In [3]:
b=Poly([1,2],x,modulus=5)
q=a//b
r=a%b
print(q)
print(r)
print(a==b*q+r)

Poly(-2*x**2 - 2*x - 2, x, modulus=5)
Poly(1, x, modulus=5)
True


## Coderen
We beschouwen een $(N,n,2e+1)_p$ Reed-Solomoncode, waarbij $N=n+2e$. De informatiewoorden zijn dus (coëfficiëntenlijsten van) veeltermen van graad $n-1$ over het veld $\mathbb{Z}_p$.

Implementeer de method $\texttt{encode(w, a, p)}$ die een informatiewoord $w$, bestaande uit $n$ elementen van het veld $\mathbb{Z}_p$ omzet in een codewoord, gegeven de lijst van $N$ interpolatiepunten $a$.

In [ ]:
def encode(w, a, p):
       #vul aan

Als je dit juist geimplementeerd hebt, zou hier $\texttt{True}$ moeten verschijnen:

In [ ]:
w = np.array([1,2,3])
a = np.array([1,2,3,4,5,6,7])
p = 11
print((encode(w,a,p)==np.array([6, 6, 1, 2, 9, 0, 8])).all())

## Inefficiënte Welch–Berlekampdecodering

(Dit deel is optioneel: als je meteen het efficiënte Welch–Berlekamp algoritme wil doen kan je dit overslaan. Het lijkt me echter eenvoudiger eerst het inefficiënte algoritme te implementeren.)

Implementeer het inefficiënte Welch–Berlekamp algoritme $\texttt{decode_inefficient(c, n, p, a)}$. Hier is $c$ het ontvangen codewoord, $n$ de lengte van de klaartextwoorden, $p$ het gekozen priemgetal, en $a$ de interpolatiepunten.

In [ ]:
def decode_inefficient(c, n, p, a):
    #vul aan

Eerste test: opgave 4 van sessie 7. Je zou $[3,3]$ moeten uitkomen.

In [ ]:
decode_inefficient([1,4,2,2],2,5,[1,2,4,8])

Tweede test: Als alles goed gegaan is zou hier nu enkel $\texttt{True}$ mogen verschijnen.

In [ ]:
w = [1,3,5,6]
p = 11
N = 8
a = [1,2,3,4,5,6,7,8]
c = encode(w, a, p)

# Decoderen zonder fouten:
print((w == decode_inefficient(c, 4, p, a)))

# Decoderen met het maximaal aantal fouten:
c[0] += 3
c[2] += 9
c %= p

print((w == decode_inefficient(c, 4, p, a)))

## Efficiënte Welch–Berlekampdecodering
Implementeer eerst het Cooley-Tukey algoritme: gegeven een vector $x$ van lengte $N$ over het veld $\mathbb{Z}_p$ en een primitieve $N$-de eenheidswortel $\alpha$, bereken de discrete Fouriertransformatie $\mathcal{F}_{\alpha}x$.

Je mag er van uit gaan dat je input wel degelijk zo gekozen is dat $\alpha$ een primitieve $N$-de eenheidswortel is, en ook dat $N$ een macht van 2 is.

In [ ]:
def cooley_tukey(alpha,x,p):
    #vul aan

Als je dit juist geimplementeerd hebt, zou hier $\texttt{True}$ moeten verschijnen:

In [ ]:
cooley_tukey(2,[0,2,0,1],5) == [3,2,2,3]

Voor de efficiënte Welch–Berlekampdecodering kiezen we $p$ een Fermat-priemgetal, $N=p-1$ (dit is een macht van 2), en $a=(1,\alpha,\alpha^2,\ldots,\alpha^{N-1})$. Implementeer eerst een functie die gegeven $\alpha$ en $p$ de interpolatiepunten $a$ berekent.

In [ ]:
def alpha_to_a(alpha,p):
    #vul aan

Nu zijn we klaar om het efficiënte Welch–Berlekampalgoritme te implementeren.

In [ ]:
def decode_efficient(c, n, p, alpha):
    #vul aan

Eerste test: opgave 4 van sessie 7.

In [ ]:
decode_efficient([1,4,2,2],2,5,2)

Met de onderstaande code kan je je algoritmes testen. De waarden van $k$ en $e$ kan je zelf kiezen, de rest zou je niet moeten veranderen. Het gekozen priemgetal is $p=2^{2^k}+1$ met $k \in \{1,2,3,4\}$. Het aantal fouten dat je kan verbeteren is $e$.

Je code laten werken voor $k=4$ (dus $p=65537$) vergt wellicht heel wat extra optimalisatie; voor dit practicum volstaat het als je implementatie werkt voor $k=3$ (dus $p=257$).

In [ ]:
#Als het met de onderstaande waarden te lang duurt, probeer eerst $k=2$ en $e=4$.
k=3
e=100

N=2**(2**k)
n=N-2*e
p=N+1
print("Dit is een ("+",".join([str(N),str(n),str(2*e+1)])+")_"+str(p)+"-code")
alpha=3
a=alpha_to_a(alpha,p)

from random import randint
w=[randint(0,N) for _ in range(n)]
print("Klaartext:")
print(w)
c=encode(w,a,p)
print("Verstuurde boodschap:")
print(c)

#We maken fouten in e willekeurige plaatsen
from random import sample
for j in sample(range(N),e):
    c[j]=randint(0,N)
print("Ontvangen boodschap:")
print(c)

print("Tijd voor efficiente decodering:")
%time d1=decode_efficient(c,n,p,alpha)
print("Efficiente decodering correct?")
print(d1==w)
print("Tijd voor inefficiente decodering:")
%time d2=decode_inefficient(c,n,p,a)
print("Inefficiente decodering correct?")
print(d2==w)